## Launch A Server

Launch the server with a reasoning model (Qwen 3.5-4B) and reasoning parser.

In [ ]:
from sglang import separate_reasoning, assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server


if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen3-4B --reasoning-parser qwen3 --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

/workspaces/sglang/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-05 17:53:32] server_args=ServerArgs(model_path='Qwen/Qwen3-4B', tokenizer_path='Qwen/Qwen3-4B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen3-4B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38475, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=376691526, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None,

Set the default backend. Note: you can set chat_template_name in RontimeEndpoint. 

In [ ]:
set_default_backend(
    RuntimeEndpoint(f"http://localhost:{port}", chat_template_name="qwen")
)

[2025-05-05 17:53:53] INFO:     127.0.0.1:37530 - "GET /get_model_info HTTP/1.1" 200 OK


Let's start with a basic question-answering task. And see how the reasoning content is generated.

In [ ]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant_begin()
    s += gen("answer", max_tokens=512)
    s += assistant_end()


state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-05-05 17:53:53] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 17:53:54] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 6.00, #queue-req: 0
[2025-05-05 17:53:54] Decode batch. #running-req: 1, #token: 104, token usage: 0.00, gen throughput (token/s): 82.06, #queue-req: 0
[2025-05-05 17:53:55] Decode batch. #running-req: 1, #token: 144, token usage: 0.00, gen throughput (token/s): 81.56, #queue-req: 0
[2025-05-05 17:53:55] Decode batch. #running-req: 1, #token: 184, token usage: 0.00, gen throughput (token/s): 81.14, #queue-req: 0
[2025-05-05 17:53:56] Decode batch. #running-req: 1, #token: 224, token usage: 0.00, gen throughput (token/s): 80.91, #queue-req: 0
[2025-05-05 17:53:56] Decode batch. #running-req: 1, #token: 264, token usage: 0.00, gen throughput (token/s): 80.55, #queue-req: 0
[2025-05-05 17:53:56] INFO:     127.0.0.1:37538 - "POST /generate HTTP/1.1" 

With `separate_reasoning`, you can move the reasoning content to `{param_name}_reasoning_content` in the state.

In [ ]:
@function
def basic_qa_separate_reasoning(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant_begin()
    s += separate_reasoning(gen("answer", max_tokens=512), model_type="qwen3")
    s += assistant_end()


reasoning_state = basic_qa_separate_reasoning("List 3 countries and their capitals.")
print_highlight(reasoning_state.stream_executor.variable_event.keys())
print_highlight(
    f"\nSeparated Reasoning Content:\n{reasoning_state['answer_reasoning_content']}"
)

print_highlight(f"\n\nContent:\n{reasoning_state['answer']}")
print_highlight(f"\n\nMessages:\n{reasoning_state.messages()[-1]}")

dict_keys(['answer', 'answer_reasoning_content'])
[2025-05-05 17:56:44] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 17:56:44] Decode batch. #running-req: 1, #token: 63, token usage: 0.00, gen throughput (token/s): 3.77, #queue-req: 0
[2025-05-05 17:56:45] Decode batch. #running-req: 1, #token: 103, token usage: 0.00, gen throughput (token/s): 82.12, #queue-req: 0
[2025-05-05 17:56:45] Decode batch. #running-req: 1, #token: 143, token usage: 0.00, gen throughput (token/s): 81.60, #queue-req: 0
[2025-05-05 17:56:46] Decode batch. #running-req: 1, #token: 183, token usage: 0.00, gen throughput (token/s): 81.17, #queue-req: 0
[2025-05-05 17:56:46] Decode batch. #running-req: 1, #token: 223, token usage: 0.00, gen throughput (token/s): 80.90, #queue-req: 0
[2025-05-05 17:56:46] INFO:     127.0.0.1:45282 - "POST /generate HTTP/1.1" 200 OK

Separated Reasoning Content:
Okay, the user is asking for three countries 

`separate_reasoning` can also be used in multi-turn conversations.

In [ ]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(
        separate_reasoning(gen("first_answer", max_tokens=512), model_type="qwen3")
    )
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(
        separate_reasoning(gen("second_answer", max_tokens=512), model_type="qwen3")
    )
    return s


reasoning_state = multi_turn_qa()
print_highlight(f"\n\nfirst_answer:\n{reasoning_state['first_answer']}")
print_highlight(
    f"\n\nfirst_answer_reasoning_content:\n{reasoning_state['first_answer_reasoning_content']}"
)
print_highlight(f"\n\nsecond_answer:\n{reasoning_state['second_answer']}")
print_highlight(
    f"\n\nsecond_answer_reasoning_content:\n{reasoning_state['second_answer_reasoning_content']}"
)

[2025-05-05 17:54:03] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 79.25, #queue-req: 0
[2025-05-05 17:54:03] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 17:54:03] Decode batch. #running-req: 1, #token: 77, token usage: 0.00, gen throughput (token/s): 75.90, #queue-req: 0
[2025-05-05 17:54:04] Decode batch. #running-req: 1, #token: 117, token usage: 0.00, gen throughput (token/s): 81.85, #queue-req: 0
[2025-05-05 17:54:04] Decode batch. #running-req: 1, #token: 157, token usage: 0.00, gen throughput (token/s): 81.36, #queue-req: 0
[2025-05-05 17:54:05] Decode batch. #running-req: 1, #token: 197, token usage: 0.00, gen throughput (token/s): 81.01, #queue-req: 0
[2025-05-05 17:54:05] Decode batch. #running-req: 1, #token: 237, token usage: 0.00, gen throughput (token/s): 80.80, #queue-req: 0
[2025-05-05 17:54:06] Decode batch. #running-req: 1, #token: 277, token usag

## Using No thinking as Qwen 3's advanced feature 

sglang separate_reasoning is particularly useful when combined with Qwen 3's advanced feature.

[Qwen 3's advanced usages](https://qwenlm.github.io/blog/qwen3/#advanced-usages)


In [ ]:
reasoning_state = basic_qa_separate_reasoning(
    "List 3 countries and their capitals. /no_think"
)
print_highlight(f"Reasoning Content:\n{reasoning_state['answer_reasoning_content']}")
print_highlight(f"Content:\n{reasoning_state['answer']}")

[2025-05-05 17:54:10] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 26, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 17:54:10] Decode batch. #running-req: 1, #token: 51, token usage: 0.00, gen throughput (token/s): 76.50, #queue-req: 0
[2025-05-05 17:54:10] INFO:     127.0.0.1:47276 - "POST /generate HTTP/1.1" 200 OK
Reasoning Content:
 
Content:
 1. France - Paris  
2. Germany - Berlin  
3. Japan - Tokyo


`separate_reasoning` can also be used in regular expression generation.

In [ ]:
@function
def regular_expression_gen(s):
    s += user(
        "What is the IP address of the Google DNS servers? just provide the answer"
    )
    s += assistant(
        separate_reasoning(
            gen(
                "answer",
                temperature=0,
                regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
                max_tokens=512,
            ),
            model_type="qwen3",
        ),
    )


reasoning_state = regular_expression_gen()

In [ ]:
print_highlight(f"Answer:\n{reasoning_state['answer']}")
print_highlight(
    f"\n\nReasoning Content:\n{reasoning_state['answer_reasoning_content']}"
)

[2025-05-05 17:54:11] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-05 17:54:11] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, gen throughput (token/s): 47.33, #queue-req: 0
[2025-05-05 17:54:12] Decode batch. #running-req: 1, #token: 108, token usage: 0.00, gen throughput (token/s): 83.03, #queue-req: 0
[2025-05-05 17:54:12] Decode batch. #running-req: 1, #token: 148, token usage: 0.00, gen throughput (token/s): 82.51, #queue-req: 0
[2025-05-05 17:54:13] Decode batch. #running-req: 1, #token: 188, token usage: 0.00, gen throughput (token/s): 82.06, #queue-req: 0
[2025-05-05 17:54:13] Decode batch. #running-req: 1, #token: 228, token usage: 0.00, gen throughput (token/s): 81.80, #queue-req: 0
[2025-05-05 17:54:14] Decode batch. #running-req: 1, #token: 268, token usage: 0.00, gen throughput (token/s): 81.48, #queue-req: 0
[2025-05-05 17:54:14] Decode batch. #running-req: 1, #token: 308, token usa